**Once your model is trained, It can be imported here and tested on all the images**

**Uncomment to get the images from imagesdatafortesting folder**

In [ ]:
# !gdown --id "imagesdatafortesting Google drive id"
# !unzip imagesdatafortesting.zip

**Download trained model and its architecture**

In [ ]:
# !gdown --id "traained model google drive model"
# !unzip /content/Model_100L_1.0_acc.zip

In [ ]:
model_path = '/content/Model_100L_1.0_acc/best_weights_with_validation_big_dataset100_acc_100.h5' #@param
model_architecture = '/content/Model_100L_1.0_acc/model_100_lay_100_acc.json' #@param

In [ ]:
# @title Libraries
from os import walk
from PIL import Image
import tensorflow as tf
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from skimage import io
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
#@title Compile the AI Model


model = tf.keras.models.model_from_json(open(model_architecture).read())

model.load_weights(model_path)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#@title Code for De Skew
def orientation(image):
    image = Image.fromarray(image)
    image = image.resize((224,224))
    image = np.expand_dims(image, axis=0)
    pred = model.predict(image/255.0,verbose = False)
    return pred
def addPadding(img):
  # print("Padding")
  if img.shape[2] > 3:
      img = img[...,:3]
  # print(img.shape)
  old_image_height, old_image_width, channels = img.shape
  # print(img.shape)
  # create new image of desired size and color (blue) for padding
  toAdd = abs(old_image_height - old_image_width)

  if old_image_width >= old_image_height:
    new_image_width = old_image_width  
    new_image_height = old_image_height + toAdd
  else:
    new_image_width = old_image_width  + toAdd
    new_image_height = old_image_height 

  if new_image_height < 1000 :new_image_height = 1000
  if new_image_width < 1200:new_image_width = 1200

  # new_image_width = old_image_width  + toAdd
  # new_image_height = old_image_height

  # print("new Dimentions")
  # print(new_image_width,new_image_height,channels)
  #  Color the padded region to white
  color = (255) * channels
  result = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)

  # compute center offset
  x_center = (new_image_width - old_image_width) // 2
  y_center = (new_image_height - old_image_height) // 2

  # copy img image into center of result image
  result[y_center:y_center+old_image_height, 
        x_center:x_center+old_image_width] = img
  # print(result.shape)

  return result


def deskew(im, max_skew=10):

    # if im.shape[2] > 3:
    #   im = im[...,:3]
    im = addPadding(im)
    
    # cv2_imshow(im)
    # '''
    height, width , _= im.shape
    # Create a grayscale image and denoise it
    im_gs = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    im_gs = cv2.fastNlMeansDenoising(im_gs, h=3)
    # print(im_gs.shape)
    
    
    # Create an inverted B&W copy using Otsu (automatic) thresholding
    im_bw = cv2.threshold(im_gs, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    # cv2_imshow(im_bw)
    # Detect lines in this image. Parameters here mostly arrived at by trial and error.
    lines = cv2.HoughLinesP(
        im_bw, 1, np.pi / 180, 200, minLineLength=width / 12, maxLineGap=width / 150
    )
    # print(lines)
    # Collect the angles of these lines (in radians)
    angles = [] 
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angles.append(np.arctan2(y2 - y1, x2 - x1))
      #  print("X1,y1", x1,y1,x2,y2)
    # If the majority of our lines are vertical, this is probably a landscape image
    landscape = np.sum([abs(angle) > np.pi / 4 for angle in angles]) > len(angles) / 2
    # Filter the angles to remove outliers based on max_skew
    if landscape:
        angles = [
            angle for angle in angles if np.deg2rad(90 - max_skew) < abs(angle) < np.deg2rad(90 + max_skew)
        ]
    else:
        angles = [angle for angle in angles if abs(angle) < np.deg2rad(max_skew)]
    if len(angles) < 5:
        # Insufficient data to deskew
        return im
    # Average the angles to a degree offset
    angle_deg = np.rad2deg(np.median(angles))
    # If this is landscape image, rotate the entire canvas appropriately
    if landscape:
        # print(angle_deg)
        if angle_deg < 0:
            im = cv2.rotate(im, cv2.ROTATE_90_CLOCKWISE)
            angle_deg += 90
        elif angle_deg > 0 and angle_deg<60:
            im = cv2.rotate(im, cv2.ROTATE_90_COUNTERCLOCKWISE)
            angle_deg -= 90
        elif angle_deg>60 and angle_deg<88.5:
          im = cv2.rotate(im, cv2.ROTATE_90_CLOCKWISE)
          angle_deg -= 90
        elif angle_deg>88.5 and angle_deg<100:
          im = cv2.rotate(im, cv2.ROTATE_90_CLOCKWISE)
          angle_deg += 90  
    # Rotate the image by the residual offset
    M = cv2.getRotationMatrix2D((width / 2, height / 2), angle_deg, 1)
    im = cv2.warpAffine(im, M, (width, height), borderMode=cv2.BORDER_REPLICATE)

    isUpsideDown = orientation(im)[0][0]


    if(isUpsideDown>=0.5):
      print("Don't rotate the image")
    else:
      print("Rotate the image")
      im = cv2.rotate(im,cv2.ROTATE_180)

    print(isUpsideDown)
    return im


**Specify the path to your image directory**

In [ ]:
path_to_image_directory = '/content/imagedirectory/' #@param
pt = path_to_image_directory

In [ ]:

filenames = next(walk(pt), (None, None, []))[2]

x = 0
for each in filenames:
  try: 
    # image = io.imread(pt+each)
    image = Image.open(pt+each)
    img = deskew(np.asarray(image))
    # cv2_imshow(img)
    f, axarr = plt.subplots(1,2,figsize=(10, 10))
    axarr[0].imshow(image)
    axarr[1].imshow(img)
    plt.show()
    # print(pt+each)
  except Exception as e:  
    print(each + " Not working #########################")   
    print(e)
  # x+=1
  # if x>=10:break